# Tạo token cho Hợp đồng Marlowe

***Trước khi chạy sổ tay này, bạn có thể muốn sử dụng chức năng "clear output" của Jupyter để xóa kết quả của lần thực thi trước của sổ tay. Điều này sẽ làm rõ hơn những gì đã được thực thi trong phiên làm việc hiện tại.***

Sổ tay này trình bày cách khắc token bản địa của Cardano sử dụng công cụ CLI của Marlowe và cách sử dụng các token như vậy trong một hợp đồng Marlowe. (Lưu ý rằng chính hợp đồng Marlowe sẽ không khắc token, nhưng chúng có thể sử dụng các token đã được khắc ở nơi khác.)

[Một video hướng dẫn qua sổ tay Jupyter này.](https://youtu.be/S0MOipqXpmQ)

Bạn có thể đặt câu hỏi về Marlowe trong [kênh #ask-marlowe trên Discord của IOG](https://discord.com/channels/826816523368005654/936295815926927390) hoặc đăng các vấn đề với bài học này vào [danh sách vấn đề của kho lưu trữ github Marlowe Starter Kit](https://github.com/input-output-hk/marlowe-starter-kit/issues).

## Điều kiện tiên quyết

Xem [Điều kiện tiên quyết](../../docs/preliminaries.md) để biết thông tin về cách thiết lập môi trường của bạn để sử dụng hướng dẫn này.

Bước đầu tiên là kiểm tra chúng ta có tất cả các công cụ cần thiết và biến môi trường có sẵn cho sổ tay không.

In [ ]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

Hãy chắc chắn rằng bạn cũng đã [thiết lập và cung cấp tài chính](../../setup/01-setup-keys.ipynb) cho các bên khác nhau
- Người phân phối token
    - `$KEYS/lender.address`: Địa chỉ Cardano cho người phân phối token
    - `$KEYS/lender.skey`: vị trí của tệp khóa ký cho người phân phối token

### Địa chỉ và quỹ của người phân phối token

Kiểm tra xem đã có địa chỉ và khóa được tạo cho người phân phối token hay chưa. Nếu chưa, xem "Tạo Địa chỉ và Khóa Ký" trong [Thiết lập Khóa](../../setup/01-setup-keys.ipynb#Creating-Addresses-and-Signing-Keys).

In [ ]:
TOKEN_DISTRIBUTOR_SKEY=$KEYS/lender.skey
TOKEN_DISTRIBUTOR_ADDR=$(cat $KEYS/lender.address)
echo "TOKEN_DISTRIBUTOR_ADDR = $TOKEN_DISTRIBUTOR_ADDR"

Bạn có thể xem địa chỉ trên một trình duyệt Cardano. Đôi khi mất khoảng ba mươi giây hoặc lâu hơn để một giao dịch được hiển thị trên trình duyệt.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $TOKEN_DISTRIBUTOR_ADDR

## Thiết kế token

Bây giờ chúng ta tạo metadata [CIP-25](https://cips.cardano.org/cips/cip25/) cho các token mà chúng ta sẽ khắc. Metadata này có khóa `721` và bao gồm những thứ sau:

- JSON mô tả token.
- Tùy chọn, liên kết đến các nguồn tài nguyên bên ngoài như hình ảnh cho token.

Metadata được lưu trữ trên blockchain trong giao dịch khắc token, nhưng hình ảnh được lưu trữ ngoài blockchain và chỉ được tham chiếu bởi metadata trên chuỗi.

Chúng ta sẽ tạo một token với tên tài sản `M4B` và kết hợp với nó metadata với một mô tả và hình ảnh.

In [ ]:
TOKEN_NAME=M4B
METADATA_FILE=marlowe-babbage.json
IMAGE_IPFS=QmZqCCHLqQcHXftNarCwKpRHbzF4mvNeQRVpzk2bdue5bw

Chúng ta sử dụng IPFS để lưu trữ hình ảnh, nhưng hình ảnh có thể được lưu trữ ở bất kỳ URI nào. Nhiều [dịch vụ ghim IPFS](https://sourceforge.net/software/ipfs-pinning/) có sẵn. Hãy tải hình ảnh, mặc dù tệp hình ảnh không yêu cầu cho quá trình khắc.

In [ ]:
IMAGE_URL=https://ipfs.io/ipfs/"$IMAGE_IPFS"
IMAGE_FILE=marlowe-babbage.png
curl -sS "$IMAGE_URL" -o "$IMAGE_FILE"

Đây là hình ảnh nó:

![Marlowe in the Babbage Era](marlowe-babbage.png)

Dưới đây là metadata cho token mà chúng ta sẽ tạo.

In [ ]:
yaml2json << EOI > "$METADATA_FILE"
$TOKEN_NAME:
  description: Marlowe smart contracts in the Babbage Era
  image: ipfs://$IMAGE_IPFS
  mediaType: image/gif
  name: Marlowe in the Babbage Era
  url: https://marlowe.iohk.io/
EOI
jq . marlowe-babbage.json

Phần trên chỉ chứa metadata cho một token duy nhất, nhưng JSON có thể có các mục cho nhiều token.

Khóa `M4B` là tên tài sản cho token trên blockchain. Các trường lồng nhau mô tả token đó.

- `name` là tên thân thiện với người dùng cho token.
- `image` là một URI tham chiếu đến vị trí của hình ảnh cho token.
- `mediaType` là kiểu MIME cho hình ảnh.
- `description` là văn bản mô tả token.
- `url` là URL để tìm thêm thông tin liên quan đến token.

Tiêu chuẩn [CIP25](https://cips.cardano.org/cips/cip25/#generalstructure) cung cấp thông tin về cấu trúc tổng quát của metadata.

## Mint token

Công cụ `marlowe-cli` hỗ trợ tạo token theo tiêu chuẩn CIP25.

In [ ]:
marlowe-cli util mint --help

Lệnh `marlowe-cli util mint` sử dụng chính sách khắc Cardano Simple Script V2. Nếu tùy chọn `--expires` được chỉ định, thì chính sách khắc sẽ bị "khóa" sau một khe thời gian được chỉ định, sao cho không còn token nào với chính sách đó có thể được khắc nữa tại hoặc sau số khe thời gian đó; nếu không, thì không có giới hạn thời gian đối với việc khắc token mới hoặc tiêu hủy token cũ.

Trước tiên, tìm số khe thời gian cho đỉnh của blockchain.

In [ ]:
TIP=$(cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | jq -r .slot)
echo "The tip is at slot number $TIP."

Hãy đặt khe thời gian hết hạn là năm giờ vào tương lai. Điều này cho chúng ta một ít thời gian để tiêu hủy và tái tạo các token nếu chúng ta đã mắc lỗi chính tả hoặc các lỗi khác trong metadata.

In [ ]:
SECONDS=1
MINUTES=$((60 * SECONDS))
HOURS=$((60 * MINUTES))
EXPIRES=$((TIP + 5 * HOURS))
echo "EXPIRES = $EXPIRES"

Chúng ta sẽ tạo ra sáu token.

In [ ]:
TOKEN_COUNT=6

Chúng ta cung cấp thông tin khóa của người phân phối token, tệp metadata, số lượng token, khe thời gian hết hạn, và điểm đến. Đổi lại, công cụ sẽ in ra ID chính sách cho kịch bản khắc.

In [ ]:
TOKEN_POLICY=$(
marlowe-cli util mint \
  --issuer "$TOKEN_DISTRIBUTOR_ADDR:$TOKEN_DISTRIBUTOR_SKEY" \
  --metadata-file "$METADATA_FILE" \
  --count "$TOKEN_COUNT" \
  --expires "$EXPIRES" \
  --out-file /dev/null \
  --submit 600s \
  "$TOKEN_NAME:$TOKEN_DISTRIBUTOR_ADDR" \
  2> /dev/null \
)
echo "TOKEN_POLICY = $TOKEN_POLICY"

Chúng ta có thể xem các token đã được khắc trên một trình duyệt:

In [ ]:
echo "$CARDANO_SCAN_URL/tokenPolicy/$TOKEN_POLICY"

## Tùy chọn: Tiêu hủy token

Marlowe CLI cũng cung cấp một lệnh để hủy bỏ (tức là "tiêu hủy") các token đã được khắc trước đó.

In [ ]:
marlowe-cli util burn --help

## Tùy chọn: Chi tiết của kịch bản tạo ra

Marlowe CLI sử dụng Simple Script V2 làm chính sách tạo ra của mình. Hãy tự mình xác định chính sách đó và xem rằng nó có cùng ID chính sách như công cụ đã báo cáo.

JSON cho kịch bản chính sách có thể được sử dụng với `cardano-cli`, tải lên các trình duyệt như CardanoScan, v.v.

Đầu tiên, chúng ta cần băm khóa công khai (PKH) của khóa ký được sử dụng để tạo ra token. Trong trường hợp của chúng ta, đó là token

In [ ]:
TOKEN_DISTRIBUTOR_PKH=$(
cardano-cli key verification-key --signing-key-file "$TOKEN_DISTRIBUTOR_SKEY" --verification-key-file /dev/stdout \
| cardano-cli address key-hash --verification-key-file /dev/stdin \
)
echo "TOKEN_DISTRIBUTOR_PKH = $TOKEN_DISTRIBUTOR_PKH"

### Chính sách tạo ra với khe thời gian hết hạn

Đối với một chính sách tạo ra có khe thời gian hết hạn, Marlowe CLI tạo ra một chính sách tiền tệ theo dạng sau:

In [ ]:
yaml2json << EOI > policy-$EXPIRES.json
type: all
scripts:
- type: sig
  keyHash: $TOKEN_DISTRIBUTOR_PKH
- type: before
  slot: $EXPIRES
EOI
jq . policy-$EXPIRES.json

Chúng ta có thể xác minh rằng chính sách này tương ứng với ID chính sách mà chúng ta đã sử dụng để tạo ra các token.

In [ ]:
cardano-cli transaction policyid --script-file "policy-$EXPIRES.json"

### Chính sách tạo ra không có thời gian hết hạn

Chính sách tạo ra được Marlowe CLI tạo ra sẽ đơn giản hơn nếu không có thời gian hết hạn.

In [ ]:
yaml2json << EOI > policy-noexpires.json
type: sig
keyHash: $TOKEN_DISTRIBUTOR_PKH
EOI
jq . policy-noexpires.json

Một lần nữa, chúng ta có thể xác minh ID chính sách của chính sách đó.

In [ ]:
cardano-cli transaction policyid --script-file policy-noexpires.json

## Ví dụ hợp đồng Marlowe: một airdrop nhỏ cho những người giữ Ada Handle

Các token mà chúng ta đã tạo ra có thể được sử dụng trong các giao dịch thông thường hoặc trong các hợp đồng Marlowe. Ở đây chúng ta sẽ sử dụng chúng trong một airdrop nhỏ cho sáu người giữ [Ada Handle](https://mint.handle.me/) tokens.

Ví dụ này minh họa cách một chính sách tiền tệ nổi tiếng như đối với [Ada Handles](https://mint.handle.me/) hoặc [Ada Domains](https://www.adadomains.io/) có thể được sử dụng như là biểu tượng tiền tệ của vai trò cho một hợp đồng Marlowe. Điều này có thể tiện lợi bởi vì bộ xác nhận thanh toán của Marlowe cung cấp *một bảo đảm trên chuỗi* rằng quỹ sẽ được giao cho người giữ token vai trò: tức là, không cần đến các dịch vụ ngoài chuỗi để xác định địa chỉ của người giữ handle hoặc domain.

Hợp đồng dưới đây chấp nhận một khoản gửi của sáu token mà chúng ta vừa tạo và sau đó phân phối mỗi người một token cho người giữ sáu Ada Handles:
- `$e.cary`
- `$f.beaumont`
- `$j.lumley`
- `$j.webster`
- `$m.herbert`
- `$w.shakespeare`

![Small airdrop contract in Marlowe](contract.png)

Lưu ý sự hiện diện của hai trường hợp `Notify` trong hợp đồng. Chúng cần thiết để chia các thanh toán thành ba giao dịch. Cố gắng thực hiện tất cả trong cùng một giao dịch như khoản gửi sẽ dẫn đến việc vượt quá chi phí thực hiện Plutus.

### Thiết kế hợp đồng

Đầu tiên, đặt thời hạn gửi token là hai mươi phút tính từ thời điểm hiện tại, được biểu diễn bằng milliseconds POSIX.

In [ ]:
DEPOSIT_DEADLINE=$((1000 * $(date -u -d "$(date) + 20 minutes" +%s)))
echo "DEPOSIT_DEADLINE = $DEPOSIT_DEADLINE POSIX milliseconds"

Ghi hợp đồng vào một tệp. (Chúng ta có thể đã chỉ tạo hợp đồng trong [Marlowe Playground](https://play.marlowe.iohk.io/) và sau đó tải xuống JSON.) *Lưu ý rằng tên vai trò không bao gồm ký tự `$` đứng trước một Ada Handle; tương tự, chúng cũng sẽ không bao gồm hậu tố `.ada` của một Ada Domain.*

In [ ]:
yaml2json << EOI > contract.json
when:
- case:
    deposits: 6
    into_account:
      address: $TOKEN_DISTRIBUTOR_ADDR
    of_token:
      currency_symbol: $TOKEN_POLICY
      token_name: $TOKEN_NAME
    party:
      address: $TOKEN_DISTRIBUTOR_ADDR
  then:
    from_account:
      address: $TOKEN_DISTRIBUTOR_ADDR
    pay: 1
    token:
      currency_symbol: $TOKEN_POLICY
      token_name: $TOKEN_NAME
    to:
      party:
        role_token: e.cary
    then:
      from_account:
        address: $TOKEN_DISTRIBUTOR_ADDR
      pay: 1
      token:
        currency_symbol: $TOKEN_POLICY
        token_name: $TOKEN_NAME
      to:
        party:
          role_token: f.beaumont
      then:
        when:
        - case:
            notify_if: true
          then:
            from_account:
              address: $TOKEN_DISTRIBUTOR_ADDR
            pay: 1
            token:
              currency_symbol: $TOKEN_POLICY
              token_name: $TOKEN_NAME
            to:
              party:
                role_token: j.lumley
            then:
              from_account:
                address: $TOKEN_DISTRIBUTOR_ADDR
              pay: 1
              token:
                currency_symbol: $TOKEN_POLICY
                token_name: $TOKEN_NAME
              to:
                party:
                  role_token: j.webster
              then:
                when:
                - case:
                    notify_if: true
                  then:
                    from_account:
                      address: $TOKEN_DISTRIBUTOR_ADDR
                    token:
                      currency_symbol: $TOKEN_POLICY
                      token_name: $TOKEN_NAME
                    to:
                      party:
                        role_token: m.herbert
                    pay: 1
                    then:
                      from_account:
                        address: $TOKEN_DISTRIBUTOR_ADDR
                      token:
                        currency_symbol: $TOKEN_POLICY
                        token_name: $TOKEN_NAME
                      pay: 1
                      to:
                        party:
                          role_token: w.shakespeare
                      then: close
                timeout: $DEPOSIT_DEADLINE
                timeout_continuation: close
        timeout: $DEPOSIT_DEADLINE
        timeout_continuation: close
timeout: $DEPOSIT_DEADLINE
timeout_continuation: close
EOI
cat contract.json

### Giao dịch 1. Tạo hợp đồng trên blockchain

Chúng ta sẽ gửi token cho người giữ [ADA Handles](https://mint.handle.me/), vì vậy chúng ta cần sử dụng ID chính sách của Ada Handles làm biểu tượng tiền tệ vai trò của Marlowe. *Nếu bạn không chạy trên `mainnet`, bạn sẽ cần sử dụng một ID chính sách khác cho token vai trò và bạn phải tự mình khắc token đó (hoặc sử dụng token vai trò đã tồn tại).

In [ ]:
ADA_HANDLES_POLICY=f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a

Bây giờ hãy để Marlowe Runtime xây dựng giao dịch để tạo hợp đồng Marlowe. Xem [Bài học 1](../01-runtime-cli) để biết một giới thiệu chi tiết hơn.

In [ ]:
CONTRACT_ID=$(
marlowe-runtime-cli create \
  --core-file contract.json \
  --role-token-policy-id "$ADA_HANDLES_POLICY" \
  --min-utxo "$((2 * 1000000))" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-1.unsigned \
| jq -r 'fromjson | .contractId' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

*Luôn kiểm tra rằng hợp đồng không có lỗi an toàn trước khi gửi giao dịch tạo nó.* Xem [Bài học 7](../07-safety) để biết thảo luận chi tiết về các kiểm tra an toàn mà Marlowe Runtime thực hiện và tại sao chúng quan trọng.

Vì hợp đồng an toàn, chúng ta ký và gửi giao dịch.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-1.unsigned \
  --timeout 600s

Sau khi giao dịch được xác nhận, chúng ta có thể xem hợp đồng trên MarloweScan:

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=info&contractId=${CONTRACT_ID/\#/%23}"

### Giao dịch 2. Gửi token và thực hiện hai airdrop đầu tiên

Đầu tiên, xây dựng giao dịch sẽ gửi token vào hợp đồng và thực hiện thanh toán cho `$e.cary` và `$f.beaumont`.

In [ ]:
TX_2=$(
marlowe-runtime-cli deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$TOKEN_DISTRIBUTOR_ADDR" \
  --to-party "$TOKEN_DISTRIBUTOR_ADDR" \
  --currency "$TOKEN_POLICY" \
  --token-name "$TOKEN_NAME" \
  --quantity "$TOKEN_COUNT" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-2.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_2 = $TX_2"

Bây giờ hãy ký và gửi nó.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-2.unsigned \
  --timeout 600s

Sau khi giao dịch được xác nhận, chúng ta có thể xem nó trên MarloweScan.

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=tx&contractId=${CONTRACT_ID/\#/%23}&transactionId=$TX_2"

### Giao dịch 3. Thông báo cho hợp đồng thực hiện hai airdrop tiếp theo

Bây giờ, hãy xây dựng giao dịch sẽ thực hiện các thanh toán cho `$j.lumley` và `$j.webster`.

In [ ]:
TX_3=$(
marlowe-runtime-cli notify \
  --contract "$CONTRACT_ID" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

Bây giờ, hãy ký và gửi nó.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-3.unsigned \
  --timeout 600s

Sau khi giao dịch được xác nhận, chúng ta có thể xem nó trên MarloweScan.

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=tx&contractId=${CONTRACT_ID/\#/%23}&transactionId=$TX_3"

### Giao dịch 4. Thông báo cho hợp đồng thực hiện hai airdrop cuối cùng

Bây giờ, hãy xây dựng giao dịch sẽ thực hiện các thanh toán cho `$m.herbert` và `$w.shakespeare`.

In [ ]:
TX_4=$(
marlowe-runtime-cli notify \
  --contract "$CONTRACT_ID" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-4.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_4 = $TX_4"

Bây giờ, hãy ký và gửi nó.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-4.unsigned \
  --timeout 600s

Sau khi giao dịch được xác nhận, chúng ta có thể xem nó trên MarloweScan.

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=tx&contractId=${CONTRACT_ID/\#/%23}&transactionId=$TX_4"

### Người nhận rút token từ địa chỉ Marlowe role-payout

Sáu khoản thanh toán đang chờ tại địa chỉ Marlowe's role-payout để những người nắm giữ mỗi Ada Handle rút chúng. Hiện tại có một số phương pháp để rút khoản thanh toán như sau:

1. Kết nối ví CIP30 của mình với ứng dụng Marlowe Payouts, sẽ liệt kê tất cả các khoản thanh toán được giữ cho lợi ích của chủ sở hữu ví.
2. Sử dụng điểm cuối rút của Marlowe Runtime REST API.
3. Sử dụng lệnh rút của Marlowe Runtime CLI.
4. Sử dụng lệnh rút của Marlowe CLI.
5. Tạo giao dịch rút bằng cách sử dụng Cardano CLI.

## Tùy chọn: Các trường hợp sử dụng liên quan đến token native và hợp đồng Marlowe

Mặc dù Marlowe Runtime hiện chưa cung cấp hỗ trợ cho việc phát hành token native kết hợp với một hợp đồng Marlowe, điều này có thể được thực hiện bằng cách sử dụng `cardano-cli transaction build`.

1. Token có thể được phát hành trong cùng một giao dịch tạo ra hợp đồng Marlowe bằng cách chạy một script minting đơn giản hoặc Plutus trong giao dịch đó.
2. Token có thể được phát hành trong cùng một giao dịch như một `Deposit` của Marlowe bằng cách chạy một script minting đơn giản hoặc Plutus trong giao dịch đó.

Lưu ý rằng việc chạy một script Simple trong cùng một giao dịch như Marlowe là giá rẻ, nhưng ít mạnh mẽ hơn so với việc chạy một script Plutus. Script Plutus có thể đảm bảo thông tin mạnh mẽ hơn về giao dịch, chẳng hạn như đảm bảo rằng việc phát hành chỉ có thể diễn ra ở một giai đoạn cụ thể của hợp đồng Marlowe.